# 🚀 Sincpro Async Worker - Demostración Interactiva

## La esencia de la librería: Ejecutar corrutinas async con subtareas paralelas desde código síncrono

Esta librería resuelve un problema común: **¿Cómo ejecutar código asíncrono desde contextos síncronos?**

### 🎯 Características principales:
- ✅ **Ejecuta corrutinas async desde código síncrono**
- ✅ **Subtareas async corren en paralelo** (como Promise.all() en JavaScript)
- ✅ **Ejecución en hilo separado** para no bloquear
- ✅ **Dos modos**: Bloqueante (esperar resultado) o Fire-and-forget (devolver Future)
- ✅ **Thread-safe** y sin configuración necesaria

---

In [ ]:
# 📦 Importar las librerías necesarias
import asyncio
import time
import concurrent.futures
from typing import List, Dict, Any
import httpx
import json

# ⭐ Importar la función principal de la librería
from sincpro_async_worker import run_async_task

print("✅ Librerías importadas correctamente!")
print("🎯 sincpro_async_worker está listo para usar")

## 🔥 Ejemplo 1: Problema Común - ¿Cómo ejecutar async desde código síncrono?

**Sin la librería, esto NO funciona en contexto síncrono:**

```python
# ❌ Esto no funciona en código síncrono
async def fetch_data():
    async with httpx.AsyncClient() as client:
        return await client.get("https://api.example.com/data")

result = await fetch_data()  # SyntaxError en contexto síncrono!
```

**Con sincpro_async_worker es súper simple:**

In [16]:
# Con sincpro_async_worker esto funciona perfectamente!

async def simple_async_task(message: str, delay: float) -> str:
    """Una tarea asincrona simple que simula trabajo."""
    print(f"Iniciando tarea: {message}")
    await asyncio.sleep(delay)
    result = f"Completado: {message} (tardo {delay}s)"
    print(result)
    return result

print("=" * 60)
print("DEMOSTRANDO sincpro_async_worker EN JUPYTER")
print("=" * 60)

print("Detectando entorno de ejecucion...")
try:
    # Verificar si estamos en un notebook
    loop = asyncio.get_running_loop()
    print("Detectado: Entorno Jupyter con event loop activo")
    
    # En Jupyter, usamos await directamente (mas natural)
    print("\nEjecutando de forma nativa en Jupyter:")
    result = await simple_async_task("Tarea nativa en Jupyter", 1.0)
    print(f"Resultado: {result}")
    
    # También podemos crear tareas y esperarlas
    print("\nCreando tarea async:")
    task = asyncio.create_task(simple_async_task("Tarea como Task", 0.8))
    result2 = await task
    print(f"Resultado de Task: {result2}")
    
except RuntimeError:
    print("Detectado: Entorno sincrono normal")
    # En entorno normal (sin event loop), usar run_async_task
    result = run_async_task(simple_async_task("Tarea en entorno normal", 1.0))
    print(f"Resultado: {result}")

print("\nLa libreria se adapta perfectamente al entorno!")
print("Nota: En Jupyter usamos await, en entornos sincronos usamos run_async_task")

DEMOSTRANDO sincpro_async_worker EN JUPYTER
Detectando entorno de ejecucion...
Detectado: Entorno Jupyter con event loop activo

Ejecutando de forma nativa en Jupyter:
Iniciando tarea: Tarea nativa en Jupyter
Completado: Tarea nativa en Jupyter (tardo 1.0s)
Resultado: Completado: Tarea nativa en Jupyter (tardo 1.0s)

Creando tarea async:
Iniciando tarea: Tarea como Task
Completado: Tarea nativa en Jupyter (tardo 1.0s)
Resultado: Completado: Tarea nativa en Jupyter (tardo 1.0s)

Creando tarea async:
Iniciando tarea: Tarea como Task
Completado: Tarea como Task (tardo 0.8s)
Resultado de Task: Completado: Tarea como Task (tardo 0.8s)

La libreria se adapta perfectamente al entorno!
Nota: En Jupyter usamos await, en entornos sincronos usamos run_async_task
Completado: Tarea como Task (tardo 0.8s)
Resultado de Task: Completado: Tarea como Task (tardo 0.8s)

La libreria se adapta perfectamente al entorno!
Nota: En Jupyter usamos await, en entornos sincronos usamos run_async_task


## 🚀 Ejemplo 2: El Poder Real - Ejecución en Paralelo (como Promise.all())

**Esta es la esencia de la librería**: Una corrutina que contiene múltiples subtareas async que se ejecutan **en paralelo** dentro del event loop, todo desde código síncrono.

### 📊 Comparación de tiempos:
- **Secuencial**: 3 tareas de 1s cada una = ~3 segundos
- **Paralelo**: 3 tareas de 1s en paralelo = ~1 segundo

¡Vamos a verlo en acción!

In [ ]:
# Ejecucion en paralelo - La esencia de la libreria

async def task_paralela(task_id: int, duration: float) -> dict:
    """Simula una tarea que toma tiempo (I/O, network, etc.)"""
    start_time = time.time()
    print(f"Tarea {task_id} iniciada")
    
    # Simular trabajo asincrono (podria ser HTTP request, DB query, etc.)
    await asyncio.sleep(duration)
    
    end_time = time.time()
    result = {
        "task_id": task_id,
        "duration": duration,
        "actual_time": round(end_time - start_time, 2),
        "status": "completada"
    }
    print(f"Tarea {task_id} completada en {result['actual_time']}s")
    return result

async def ejecutar_tareas_en_paralelo() -> list:
    """
    ESTA ES LA MAGIA: 
    Multiples tareas async corriendo en paralelo dentro de una corrutina
    """
    print("Iniciando 5 tareas en paralelo...")
    
    # Crear las tareas (como Promise.all() en JavaScript)
    tasks = [
        task_paralela(1, 1.0),
        task_paralela(2, 1.5),
        task_paralela(3, 0.8),
        task_paralela(4, 1.2),
        task_paralela(5, 0.9)
    ]
    
    # Ejecutar TODAS las tareas en paralelo
    start_total = time.time()
    resultados = await asyncio.gather(*tasks)
    end_total = time.time()
    
    print(f"\nTodas las tareas completadas en {round(end_total - start_total, 2)}s!")
    print(f"Si hubieran sido secuenciales: {sum([1.0, 1.5, 0.8, 1.2, 0.9])}s")
    
    return resultados

# Ejecutar en Jupyter (usando await directo)
start_time = time.time()

# En Jupyter usamos await directamente
resultados = await ejecutar_tareas_en_paralelo()

end_time = time.time()
total_time = round(end_time - start_time, 2)

print(f"\nRESULTADO FINAL:")
print(f"   Tiempo total: {total_time} segundos")
print(f"   Tareas completadas: {len(resultados)}")
print(f"   Ejecutado en event loop de Jupyter!")

# Mostrar resumen de resultados
for resultado in resultados:
    print(f"   • Tarea {resultado['task_id']}: {resultado['actual_time']}s")

## 🌐 Ejemplo 3: Caso Real - HTTP Requests en Paralelo

**Caso de uso común**: Necesitas hacer múltiples llamadas a APIs desde tu aplicación síncrona.

### Sin paralelismo:
- 5 requests de ~1s cada uno = ~5 segundos total

### Con sincpro_async_worker:
- 5 requests en paralelo = ~1-2 segundos total

¡Vamos a probarlo con APIs reales!

In [ ]:
# 🌐 HTTP Requests en paralelo - Caso de uso real

async def fetch_api(client: httpx.AsyncClient, url: str, name: str) -> Dict[str, Any]:
    """Hacer request a una API y medir el tiempo"""
    start_time = time.time()
    print(f"🌐 Fetching {name}...")
    
    try:
        response = await client.get(url, timeout=10.0)
        end_time = time.time()
        
        result = {
            "name": name,
            "url": url,
            "status_code": response.status_code,
            "response_time": round(end_time - start_time, 2),
            "content_length": len(response.text),
            "success": True
        }
        print(f"✅ {name} completado: {result['status_code']} ({result['response_time']}s)")
        return result
        
    except Exception as e:
        end_time = time.time()
        print(f"❌ {name} falló: {str(e)}")
        return {
            "name": name,
            "url": url,
            "error": str(e),
            "response_time": round(end_time - start_time, 2),
            "success": False
        }

async def hacer_requests_paralelos() -> List[Dict[str, Any]]:
    """
    🎯 PARALELO: Múltiples HTTP requests simultáneos
    Esto es lo que hace la librería especial - paralelismo real!
    """
    apis = [
        ("https://httpbin.org/delay/1", "API 1 (delay 1s)"),
        ("https://httpbin.org/json", "API 2 (JSON)"),
        ("https://httpbin.org/user-agent", "API 3 (User-Agent)"),
        ("https://httpbin.org/headers", "API 4 (Headers)"),
        ("https://httpbin.org/delay/2", "API 5 (delay 2s)")
    ]
    
    print("🚀 Iniciando requests HTTP en paralelo...")
    start_total = time.time()
    
    async with httpx.AsyncClient() as client:
        # 🔥 Crear todas las tareas para ejecución paralela
        tasks = [fetch_api(client, url, name) for url, name in apis]
        
        # ✨ Ejecutar TODOS los requests en paralelo
        resultados = await asyncio.gather(*tasks, return_exceptions=True)
    
    end_total = time.time()
    total_time = round(end_total - start_total, 2)
    
    print(f"\n🎉 Todos los requests completados en {total_time}s!")
    
    # Limpiar excepciones que puedan haber quedado
    resultados_limpios = []
    for resultado in resultados:
        if isinstance(resultado, dict):
            resultados_limpios.append(resultado)
        else:
            print(f"⚠️  Excepción capturada: {resultado}")
    
    return resultados_limpios

# 🚀 Ejecutar desde código síncrono
print("=" * 60)
print("🌐 DEMOSTRACION DE HTTP REQUESTS PARALELOS")
print("=" * 60)

start_time = time.time()
resultados_http = run_async_task(hacer_requests_paralelos(), timeout=30.0)
end_time = time.time()

print(f"\n📊 RESUMEN DE RESULTADOS:")
print(f"   ⏱️  Tiempo total desde código síncrono: {round(end_time - start_time, 2)}s")
print(f"   🌐 Requests completados: {len([r for r in resultados_http if r.get('success', False)])}")
print(f"   ❌ Requests fallidos: {len([r for r in resultados_http if not r.get('success', True)])}")

# Mostrar detalles
for resultado in resultados_http:
    if resultado.get('success', False):
        print(f"   ✅ {resultado['name']}: {resultado['status_code']} en {resultado['response_time']}s")
    else:
        print(f"   ❌ {resultado['name']}: Error en {resultado['response_time']}s")

## 🔥 Ejemplo 4: Modo Fire-and-Forget - Ejecución No Bloqueante

**Los dos modos de ejecución:**

### 🔒 Modo Bloqueante (por defecto):
```python
resultado = run_async_task(mi_corrutina())  # Espera hasta completar
```

### 🚀 Modo Fire-and-Forget:
```python
future = run_async_task(mi_corrutina(), fire_and_forget=True)  # Retorna inmediatamente
# ... hacer otro trabajo ...
resultado = future.result(timeout=30)  # Obtener resultado cuando esté listo
```

¡Vamos a ver ambos modos en acción!

In [ ]:
# 🔥 Comparación de modos: Bloqueante vs Fire-and-Forget

async def tarea_larga() -> Dict[str, Any]:
    """Simula una tarea que toma tiempo considerable"""
    print("🕐 Iniciando tarea larga...")
    
    # Simular múltiples operaciones en paralelo
    tasks = []
    for i in range(3):
        async def operacion(idx):
            await asyncio.sleep(1.5)
            return f"Operación {idx} completada"
        tasks.append(operacion(i))
    
    resultados = await asyncio.gather(*tasks)
    
    resultado_final = {
        "operaciones": resultados,
        "tiempo_total": "~1.5s (paralelo)",
        "timestamp": time.time()
    }
    
    print("✅ Tarea larga completada!")
    return resultado_final

print("=" * 70)
print("🔒 MODO BLOQUEANTE (espera hasta completar)")
print("=" * 70)

# 🔒 Modo bloqueante
print("⏳ Ejecutando en modo bloqueante...")
start_blocking = time.time()

resultado_bloqueante = run_async_task(tarea_larga())

end_blocking = time.time()
print(f"✅ Modo bloqueante completado en {round(end_blocking - start_blocking, 2)}s")
print(f"📊 Resultado: {len(resultado_bloqueante['operaciones'])} operaciones")

print("\n" + "=" * 70)
print("🚀 MODO FIRE-AND-FORGET (no bloquea)")
print("=" * 70)

# 🚀 Modo fire-and-forget
print("🚀 Iniciando tarea en background...")
start_ff = time.time()

# Iniciar tarea en background
future = run_async_task(tarea_larga(), fire_and_forget=True)

# ¡Inmediatamente podemos hacer otro trabajo!
launch_time = time.time()
print(f"⚡ Tarea lanzada en {round(launch_time - start_ff, 3)}s (¡súper rápido!)")

# Simular trabajo adicional mientras la tarea corre en background
print("💼 Haciendo otro trabajo mientras la tarea corre en background...")
for i in range(3):
    time.sleep(0.3)
    print(f"   💼 Trabajo adicional paso {i+1}...")

print("🔍 Verificando si la tarea background está lista...")

# Obtener el resultado cuando esté listo
resultado_ff = future.result(timeout=10)  # Esperar máximo 10s
end_ff = time.time()

print(f"✅ Tarea background completada!")
print(f"📊 Tiempo total: {round(end_ff - start_ff, 2)}s")
print(f"🎯 Pudimos hacer otro trabajo mientras la tarea corría!")

# Verificar que es el mismo tipo de Future que devuelve
print(f"\n🔍 Tipo de Future: {type(future)}")
print(f"🔍 Es concurrent.futures.Future? {isinstance(future, concurrent.futures.Future)}")

print("\n🎉 ¡Los dos modos funcionan perfectamente!")

## ⚠️ Ejemplo 5: Manejo de Errores y Timeouts

La librería maneja errores elegantemente:

- ✅ **Excepciones de corrutinas** se propagan normalmente
- ✅ **Timeouts** se manejan correctamente  
- ✅ **Errores de red** se capturan y propagan
- ✅ **Graceful degradation** cuando hay problemas internos

¡Vamos a probar diferentes escenarios de error!

In [ ]:
# ⚠️ Manejo de errores y timeouts

async def tarea_que_falla():
    """Tarea que siempre falla para probar manejo de errores"""
    print("💥 Esta tarea va a fallar...")
    await asyncio.sleep(0.5)
    raise ValueError("¡Algo salió mal en la corrutina!")

async def tarea_muy_lenta():
    """Tarea que toma mucho tiempo - para probar timeouts"""
    print("🐌 Esta tarea es muy lenta...")
    await asyncio.sleep(10)  # 10 segundos
    return "Finalmente terminé"

print("=" * 60)
print("⚠️  PROBANDO MANEJO DE ERRORES")
print("=" * 60)

# Caso 1: Excepción en la corrutina
print("🧪 Caso 1: Excepción en corrutina")
try:
    resultado = run_async_task(tarea_que_falla())
    print(f"✅ Resultado: {resultado}")
except ValueError as e:
    print(f"✅ Excepción capturada correctamente: {e}")
except Exception as e:
    print(f"❌ Excepción inesperada: {e}")

print()

# Caso 2: Timeout
print("🧪 Caso 2: Timeout (configurado a 2s, tarea toma 10s)")
try:
    resultado = run_async_task(tarea_muy_lenta(), timeout=2.0)
    print(f"✅ Resultado: {resultado}")
except TimeoutError as e:
    print(f"✅ Timeout capturado correctamente: {e}")
except Exception as e:
    print(f"❌ Excepción inesperada: {e}")

print()

# Caso 3: Error en fire-and-forget mode
print("🧪 Caso 3: Error en modo fire-and-forget")
try:
    future = run_async_task(tarea_que_falla(), fire_and_forget=True)
    print("🚀 Future creado, esperando resultado...")
    resultado = future.result(timeout=5.0)
    print(f"✅ Resultado: {resultado}")
except ValueError as e:
    print(f"✅ Excepción capturada del Future: {e}")
except Exception as e:
    print(f"❌ Excepción inesperada: {e}")

print()

# Caso 4: Múltiples tareas, algunas fallan
async def tareas_mixtas():
    """Algunas tareas exitosas, otras fallan"""
    print("🎯 Ejecutando tareas mixtas (algunas fallan)...")
    
    async def tarea_exitosa(id_tarea):
        await asyncio.sleep(0.5)
        return f"Tarea {id_tarea} exitosa"
    
    async def tarea_falla(id_tarea):
        await asyncio.sleep(0.3)
        raise RuntimeError(f"Tarea {id_tarea} falló")
    
    # Usar return_exceptions=True para capturar errores sin parar todo
    resultados = await asyncio.gather(
        tarea_exitosa(1),
        tarea_falla(2),
        tarea_exitosa(3),
        tarea_falla(4),
        return_exceptions=True
    )
    
    return resultados

print("🧪 Caso 4: Tareas paralelas con algunas que fallan")
try:
    resultados_mixtos = run_async_task(tareas_mixtas())
    print("✅ Resultados mixtos obtenidos:")
    for i, resultado in enumerate(resultados_mixtos, 1):
        if isinstance(resultado, Exception):
            print(f"   ❌ Tarea {i}: {resultado}")
        else:
            print(f"   ✅ Tarea {i}: {resultado}")
except Exception as e:
    print(f"❌ Error inesperado: {e}")

print("\n🎉 ¡Manejo de errores funciona correctamente!")

## 🎯 Resumen: La Esencia de sincpro_async_worker

### ✅ **Lo que acabamos de demostrar:**

1. **🚀 Ejecución de async desde síncrono**: Resolver el problema común de contextos mixtos
2. **⚡ Paralelismo real**: Múltiples subtareas async corriendo en paralelo (como Promise.all())
3. **🧵 Hilo separado**: Sin bloquear el hilo principal
4. **🔄 Dos modos**: Bloqueante (esperar resultado) y Fire-and-forget (Future)
5. **🌐 Casos reales**: HTTP requests, APIs, I/O operations
6. **⚠️ Manejo robusto**: Errores, timeouts y excepciones manejados elegantemente

### 🎯 **Casos de uso ideales:**
- **APIs REST** que necesitan llamar servicios async
- **Aplicaciones síncronas** que necesitan I/O asíncrono
- **Integración de librerías async** en código legacy
- **Mejora de performance** con paralelismo

### 🚫 **No usar para:**
- Sistemas distribuidos (usa Celery, RabbitMQ)
- Tareas CPU intensivas (usa multiprocessing)
- Necesidades de persistencia (usa colas de mensajes)

---

## 🏆 **Conclusión**

**sincpro_async_worker** es la solución perfecta para ejecutar corrutinas asíncronas con subtareas paralelas desde código síncrono, todo en un hilo separado y sin configuración compleja.

¡Simple, poderoso y eficiente! 🎉